# MetricsTracker Demo (TT-31 + TT-37)

Demonstrates the position metrics engine joining live DXLink Quote and Greeks
data with account positions. Loads positions via REST, subscribes to DXLink for
streamer symbols, and displays the joined DataFrame with live Greeks for options
and theoretical defaults for delta-1 instruments.

In [1]:
import asyncio
import logging
import os

import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown, display

from tastytrade.accounts import AccountsClient
from tastytrade.analytics.metrics import DELTA_1_TYPES, OPTION_TYPES, MetricsTracker
from tastytrade.config import RedisConfigManager
from tastytrade.config.enumerations import Channels
from tastytrade.connections import Credentials
from tastytrade.connections.requests import AsyncSessionHandler
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.messaging.processors.metrics import MetricsEventProcessor

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", 40)

load_dotenv("/workspace/.env", override=True)

logging.basicConfig(level=logging.INFO)
logging.getLogger("websockets").setLevel(logging.WARNING)

_OBFUSCATE = os.getenv("OBFUSCATE_ACCOUNTS", "false").lower() == "true"


def mask(account_number: str) -> str:
    if not _OBFUSCATE or len(account_number) <= 4:
        return account_number
    return "***" + account_number[-4:]


print(f"Account obfuscation: {'ON' if _OBFUSCATE else 'OFF'}")

Account obfuscation: ON


# 1. Connect & Fetch Positions

In [2]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")
session = await AsyncSessionHandler.create(credentials)
client = AccountsClient(session)

account = credentials.account_number
positions = await client.get_positions(account)

display(Markdown(f"**{len(positions)} positions** in account {mask(account)}"))

pos_summary = [{
    "Symbol": p.symbol,
    "Type": p.instrument_type.value,
    "Qty": p.quantity,
    "Direction": p.quantity_direction.value,
    "Streamer": p.streamer_symbol,
} for p in positions]
display(pd.DataFrame(pos_summary))

INFO:tastytrade.config.manager:Initialized 52 variables from .env file in Redis
INFO:tastytrade.connections.requests:Session created successfully
INFO:tastytrade.accounts.client:Fetched 12 positions for account 5WY89822


**12 positions** in account ***9822

,Symbol,Type,Qty,Direction,Streamer
0,./6EM6 EUUJ6 260403C1.225,Future Option,1.00,Short,./EUUJ26C1.225:XCME
1,./CLJ6 LOJ6 260317P56,Future Option,1.00,Short,./LOJ26P56:XNYM
2,./CLJ6 LOJ6 260317C85,Future Option,1.00,Short,./LOJ26C85:XNYM
3,./MESM6EX3H6 260320P6450,Future Option,3.00,Short,./EX3H26P6450:XCME
4,MCD 260320P00305000,Equity Option,1.00,Short,.MCD260320P305
5,MCD 260320P00295000,Equity Option,1.00,Long,.MCD260320P295
6,./MESM6EX3H6 260320C7275,Future Option,3.00,Short,./EX3H26C7275:XCME
7,./6EM6 EUUJ6 260403P1.16,Future Option,1.00,Short,./EUUJ26P1.16:XCME
8,CSCO 260227C00078000,Equity Option,1.00,Short,.CSCO260227C78
9,SPY,Equity,100.29,Long,SPY


# 2. Initialize MetricsTracker

In [3]:
tracker = MetricsTracker()
tracker.load_positions(positions)

streamer_symbols = tracker.get_streamer_symbols()
option_symbols = tracker.get_option_streamer_symbols()
skipped = [p.symbol for p in positions if p.streamer_symbol is None]

display(Markdown(f"**Tracking {len(tracker.securities)} securities** ({len(skipped)} skipped — no streamer symbol)"))
display(Markdown(f"**{len(option_symbols)} option symbols** for Greeks channel: `{sorted(option_symbols)}`"))
if skipped:
    print(f"Skipped: {skipped}")

display(Markdown("**Initial state (no market data yet):**"))
display(tracker.df[["symbol", "instrument_type", "quantity", "quantity_direction",
                     "bid_price", "ask_price", "mid_price",
                     "delta", "gamma", "theta", "vega"]])

INFO:tastytrade.analytics.metrics:Loaded 12 positions (12 with streamer symbols)


**Tracking 12 securities** (1 skipped — no streamer symbol)

**9 option symbols** for Greeks channel: `['./EUUJ26C1.225:XCME', './EUUJ26P1.16:XCME', './EX3H26C7275:XCME', './EX3H26P6450:XCME', './LOJ26C85:XNYM', './LOJ26P56:XNYM', '.CSCO260227C78', '.MCD260320P295', '.MCD260320P305']`

Skipped: ['CSCO']


**Initial state (no market data yet):**

,symbol,instrument_type,quantity,quantity_direction,bid_price,ask_price,mid_price,delta,gamma,theta,vega
0,./6EM6 EUUJ6 260403C1.225,InstrumentType.FUTURE_OPTION,1.00,QuantityDirection.SHORT,None,None,None,NaN,NaN,NaN,NaN
1,./CLJ6 LOJ6 260317P56,InstrumentType.FUTURE_OPTION,1.00,QuantityDirection.SHORT,None,None,None,NaN,NaN,NaN,NaN
2,./CLJ6 LOJ6 260317C85,InstrumentType.FUTURE_OPTION,1.00,QuantityDirection.SHORT,None,None,None,NaN,NaN,NaN,NaN
3,./MESM6EX3H6 260320P6450,InstrumentType.FUTURE_OPTION,3.00,QuantityDirection.SHORT,None,None,None,NaN,NaN,NaN,NaN
4,MCD 260320P00305000,InstrumentType.EQUITY_OPTION,1.00,QuantityDirection.SHORT,None,None,None,NaN,NaN,NaN,NaN
5,MCD 260320P00295000,InstrumentType.EQUITY_OPTION,1.00,QuantityDirection.LONG,None,None,None,NaN,NaN,NaN,NaN
6,./MESM6EX3H6 260320C7275,InstrumentType.FUTURE_OPTION,3.00,QuantityDirection.SHORT,None,None,None,NaN,NaN,NaN,NaN
7,./6EM6 EUUJ6 260403P1.16,InstrumentType.FUTURE_OPTION,1.00,QuantityDirection.SHORT,None,None,None,NaN,NaN,NaN,NaN
8,CSCO 260227C00078000,InstrumentType.EQUITY_OPTION,1.00,QuantityDirection.SHORT,None,None,None,NaN,NaN,NaN,NaN
9,SPY,InstrumentType.EQUITY,100.29,QuantityDirection.LONG,None,None,None,1.0,0.0,0.0,0.0


# 3. Open DXLink & Attach MetricsEventProcessor

In [4]:
dxlink = DXLinkManager()
await dxlink.open(credentials=credentials)

router = dxlink.router
if router is None:
    raise RuntimeError("DXLink router not initialized")

processor = MetricsEventProcessor(tracker)

# Attach to Quote handler (prices)
quote_handler = router.handler.get(Channels.Quote)
if quote_handler is None:
    raise RuntimeError("Quote handler not found")
quote_handler.add_processor(processor)

# Attach to Greeks handler (delta, gamma, theta, vega, rho, IV)
greeks_handler = router.handler.get(Channels.Greeks)
if greeks_handler is None:
    raise RuntimeError("Greeks handler not found")
greeks_handler.add_processor(processor)

print(f"MetricsEventProcessor attached to Quote and Greeks handlers")
print(f"Quote handler processors: {list(quote_handler.processors.keys())}")
print(f"Greeks handler processors: {list(greeks_handler.processors.keys())}")

INFO:tastytrade.connections.requests:Session created successfully
INFO:tastytrade.messaging.handlers:Started Channels.Control listener on channel 0
INFO:tastytrade.messaging.handlers:Started Channels.Quote listener on channel 7
INFO:tastytrade.messaging.handlers:Started Channels.Trade listener on channel 5
INFO:tastytrade.messaging.handlers:Started Channels.Greeks listener on channel 11
INFO:tastytrade.messaging.handlers:Started Channels.Profile listener on channel 1
INFO:tastytrade.messaging.handlers:Started Channels.Summary listener on channel 3
INFO:tastytrade.messaging.handlers:Started Channels.Candle listener on channel 9


MetricsEventProcessor attached to Quote and Greeks handlers
Quote handler processors: ['feed', 'metrics']
Greeks handler processors: ['feed', 'metrics']


INFO:tastytrade.messaging.handlers:SETUP
INFO:tastytrade.messaging.handlers:AUTH_STATE:AUTHORIZED
INFO:tastytrade.messaging.handlers:CHANNEL_OPENED:1
INFO:tastytrade.messaging.handlers:CHANNEL_OPENED:3
INFO:tastytrade.messaging.handlers:CHANNEL_OPENED:5
INFO:tastytrade.messaging.handlers:CHANNEL_OPENED:7
INFO:tastytrade.messaging.handlers:CHANNEL_OPENED:9
INFO:tastytrade.messaging.handlers:CHANNEL_OPENED:11
INFO:tastytrade.messaging.handlers:CHANNEL_OPENED:99
INFO:tastytrade.messaging.handlers:FEED_CONFIG:5:COMPACT
INFO:tastytrade.messaging.handlers:FEED_CONFIG:7:COMPACT
INFO:tastytrade.messaging.handlers:FEED_CONFIG:11:COMPACT
INFO:tastytrade.messaging.handlers:FEED_CONFIG:1:COMPACT
INFO:tastytrade.messaging.handlers:FEED_CONFIG:3:COMPACT
INFO:tastytrade.messaging.handlers:FEED_CONFIG:9:COMPACT
INFO:tastytrade.messaging.handlers:FEED_CONFIG:5:COMPACT:SUBSCRIBED
INFO:tastytrade.messaging.handlers:FEED_CONFIG:7:COMPACT:SUBSCRIBED
INFO:tastytrade.messaging.handlers:FEED_CONFIG:11:COMPACT

# 4. Subscribe to Position Streamer Symbols

In [5]:
symbols_list = sorted(streamer_symbols)
print(f"Subscribing to {len(symbols_list)} symbols:")
for s in symbols_list:
    print(f"  {s}")

await dxlink.subscribe(symbols_list)
print(f"\nSubscribed. Waiting for quotes to flow...")

INFO:tastytrade.connections.sockets:Added subscription: ./EUUJ26C1.225:XCME
INFO:tastytrade.connections.sockets:Added subscription: ./EUUJ26P1.16:XCME
INFO:tastytrade.connections.sockets:Added subscription: ./EX3H26C7275:XCME
INFO:tastytrade.connections.sockets:Added subscription: ./EX3H26P6450:XCME
INFO:tastytrade.connections.sockets:Added subscription: ./LOJ26C85:XNYM
INFO:tastytrade.connections.sockets:Added subscription: ./LOJ26P56:XNYM
INFO:tastytrade.connections.sockets:Added subscription: .CSCO260227C78
INFO:tastytrade.connections.sockets:Added subscription: .MCD260320P295
INFO:tastytrade.connections.sockets:Added subscription: .MCD260320P305
INFO:tastytrade.connections.sockets:Added subscription: CSCO
INFO:tastytrade.connections.sockets:Added subscription: QQQ
INFO:tastytrade.connections.sockets:Added subscription: SPY


Subscribing to 12 symbols:
  ./EUUJ26C1.225:XCME
  ./EUUJ26P1.16:XCME
  ./EX3H26C7275:XCME
  ./EX3H26P6450:XCME
  ./LOJ26C85:XNYM
  ./LOJ26P56:XNYM
  .CSCO260227C78
  .MCD260320P295
  .MCD260320P305
  CSCO
  QQQ
  SPY

Subscribed. Waiting for quotes to flow...


# 5. View Live Metrics

Re-run this cell to see updated prices and Greeks as data arrives.

In [7]:
await asyncio.sleep(3)  # allow quotes to arrive

df = tracker.df

# Reorder columns for readability
display_cols = [
    "symbol", "instrument_type", "quantity", "quantity_direction",
    "bid_price", "ask_price", "mid_price",
    "delta", "gamma", "theta", "vega", "implied_volatility",
    "price_updated_at",
]
available_cols = [c for c in display_cols if c in df.columns]

populated = df["bid_price"].notna().sum()
display(Markdown(f"**{populated}/{len(df)} securities** have live quote data"))
display(df[available_cols])

**12/12 securities** have live quote data

,symbol,instrument_type,quantity,quantity_direction,bid_price,ask_price,mid_price,delta,gamma,theta,vega,implied_volatility,price_updated_at
0,./6EM6 EUUJ6 260403C1.225,InstrumentType.FUTURE_OPTION,1.00,QuantityDirection.SHORT,0.00,0.00,0.00,0.23,8.31,-0.00,0.00,0.08,2026-02-10 05:49:09.560091
1,./CLJ6 LOJ6 260317P56,InstrumentType.FUTURE_OPTION,1.00,QuantityDirection.SHORT,0.55,0.58,0.56,-0.13,0.03,-0.02,0.04,0.40,2026-02-10 05:48:22.096016
2,./CLJ6 LOJ6 260317C85,InstrumentType.FUTURE_OPTION,1.00,QuantityDirection.SHORT,0.77,0.80,0.79,0.12,0.01,-0.04,0.04,0.71,2026-02-10 05:49:14.595555
3,./MESM6EX3H6 260320P6450,InstrumentType.FUTURE_OPTION,3.00,QuantityDirection.SHORT,28.00,28.75,28.38,-0.11,0.00,-1.25,4.30,0.23,2026-02-10 05:48:55.095638
4,MCD 260320P00305000,InstrumentType.EQUITY_OPTION,1.00,QuantityDirection.SHORT,2.23,2.46,2.34,-0.18,0.01,-0.08,0.28,0.22,2026-02-10 05:44:38.510331
5,MCD 260320P00295000,InstrumentType.EQUITY_OPTION,1.00,QuantityDirection.LONG,1.11,1.43,1.27,-0.10,0.01,-0.06,0.19,0.24,2026-02-10 05:44:38.510037
6,./MESM6EX3H6 260320C7275,InstrumentType.FUTURE_OPTION,3.00,QuantityDirection.SHORT,26.50,27.00,26.75,0.19,0.00,-0.91,6.19,0.11,2026-02-10 05:48:44.092131
7,./6EM6 EUUJ6 260403P1.16,InstrumentType.FUTURE_OPTION,1.00,QuantityDirection.SHORT,0.00,0.00,0.00,-0.11,5.91,-0.00,0.00,0.07,2026-02-10 05:47:45.560130
8,CSCO 260227C00078000,InstrumentType.EQUITY_OPTION,1.00,QuantityDirection.SHORT,9.20,10.10,9.65,0.86,0.02,-0.06,0.04,0.48,2026-02-10 05:44:38.509147
9,SPY,InstrumentType.EQUITY,100.29,QuantityDirection.LONG,694.19,694.29,694.24,1.00,0.00,0.00,0.00,NaN,2026-02-10 05:49:08.712121


# 6. Delta-1 vs Options Greeks Comparison

Delta-1 products have theoretical defaults (delta=+/-1, gamma/theta/vega/rho=0).
Options have live Greeks populated from the DXLink Greeks channel.

In [8]:
df = tracker.df

delta1_mask = df["instrument_type"].isin([t.value for t in DELTA_1_TYPES])
option_mask = df["instrument_type"].isin([t.value for t in OPTION_TYPES])

greeks_cols = ["symbol", "instrument_type", "delta", "gamma", "theta", "vega", "rho", "implied_volatility", "greeks_updated_at"]

display(Markdown("**Delta-1 positions** (theoretical Greeks):"))
display(df.loc[delta1_mask, greeks_cols])

display(Markdown("**Option positions** (live Greeks from DXLink):"))
display(df.loc[option_mask, greeks_cols])

**Delta-1 positions** (theoretical Greeks):

,symbol,instrument_type,delta,gamma,theta,vega,rho,implied_volatility,greeks_updated_at
9,SPY,InstrumentType.EQUITY,1.0,0.0,0.0,0.0,0.0,NaN,2026-02-10 05:44:38.003356
10,CSCO,InstrumentType.EQUITY,1.0,0.0,0.0,0.0,0.0,NaN,2026-02-10 05:44:38.003356
11,QQQ,InstrumentType.EQUITY,1.0,0.0,0.0,0.0,0.0,NaN,2026-02-10 05:44:38.003356


**Option positions** (live Greeks from DXLink):

,symbol,instrument_type,delta,gamma,theta,vega,rho,implied_volatility,greeks_updated_at
0,./6EM6 EUUJ6 260403C1.225,InstrumentType.FUTURE_OPTION,0.23,8.31,-0.00,0.00,0.00,0.08,2026-02-10 05:49:59.060958
1,./CLJ6 LOJ6 260317P56,InstrumentType.FUTURE_OPTION,-0.13,0.03,-0.02,0.04,-0.01,0.40,2026-02-10 05:49:49.096892
2,./CLJ6 LOJ6 260317C85,InstrumentType.FUTURE_OPTION,0.12,0.01,-0.04,0.04,0.01,0.71,2026-02-10 05:49:49.096290
3,./MESM6EX3H6 260320P6450,InstrumentType.FUTURE_OPTION,-0.11,0.00,-1.25,4.30,-0.85,0.23,2026-02-10 05:49:53.596006
4,MCD 260320P00305000,InstrumentType.EQUITY_OPTION,-0.18,0.01,-0.08,0.28,-0.07,0.22,2026-02-10 05:44:38.512959
5,MCD 260320P00295000,InstrumentType.EQUITY_OPTION,-0.10,0.01,-0.06,0.19,-0.04,0.24,2026-02-10 05:44:38.512233
6,./MESM6EX3H6 260320C7275,InstrumentType.FUTURE_OPTION,0.19,0.00,-0.91,6.19,1.38,0.11,2026-02-10 05:48:41.092108
7,./6EM6 EUUJ6 260403P1.16,InstrumentType.FUTURE_OPTION,-0.11,5.91,-0.00,0.00,-0.00,0.07,2026-02-10 05:49:59.061546
8,CSCO 260227C00078000,InstrumentType.EQUITY_OPTION,0.86,0.02,-0.06,0.04,0.03,0.48,2026-02-10 05:44:38.512550


# 7. Cleanup

In [9]:
await dxlink.close()
await session.close()
print("All connections closed.")

INFO:tastytrade.connections.sockets:Listener task cancelled
INFO:tastytrade.connections.sockets:Keepalive task cancelled
INFO:tastytrade.connections.routing:Initiating cleanup...
INFO:tastytrade.connections.routing:Cleanup completed
INFO:tastytrade.connections.requests:Session closed
INFO:tastytrade.connections.sockets:Connection closed and cleaned up
INFO:tastytrade.connections.requests:Session closed


All connections closed.
